In [6]:
!pip install selenium
from selenium import webdriver
#!pip install BeautifulSoup
from bs4 import BeautifulSoup
import os
import io
import time
import re
import random
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select



In [9]:
# Save a csv for Haoche to visualize 
# Changes - every 5000 we save it. _1 _2 (port entry name, year)
# Year search
# Try - catch (save where it failed, keep going)

#### INPUTS #####
# Change depending on which one you want to scrape
immigrant_page = "https://stevemorse.org/ellis/passengers.php?mode=ny"

# My Credentials
user = "seanlikesblue@gmail.com"
password = "pointsunknown123"

# Year to input:
year = '1887'
port_entry = "NY"

#### Scraping Code #####
# Open log in page
driver = webdriver.Chrome(executable_path="./chromedriver_mac64/chromedriver")
page = driver.get('https://www.ancestry.com/account/signin?returnUrl=')

# Log in
wait = WebDriverWait(driver, 2)
element = wait.until(EC.presence_of_element_located((By.ID, 'username')))
element.click()
element.send_keys(user)
element = wait.until(EC.presence_of_element_located((By.ID, 'password')))
element.click()
element.send_keys(password)

element = wait.until(EC.presence_of_element_located((By.ID, 'signInBtn')))
element.click()
time.sleep(5)
driver.get(immigrant_page) # INPUT 

# Select the year
year_select = Select(driver.find_element(By.NAME, 'yr'))
year_select.select_by_value(year)

# Select the yrd
yrd_input = driver.find_element(By.NAME, 'yrd')
yrd_input.send_keys('5')

# Find the search button
element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="button"][value="search"]')))
time.sleep(3)
element.click()

# Switch to the opened tab
p = driver.current_window_handle
parent = driver.window_handles[0]
chld = driver.window_handles[1]
driver.switch_to.window(chld)

# Begin Iterating and Saving Dataframes:
counter = 0
data = []
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
max_element = soup.find('span', {'class': 'pagingMax'})
max_value = int(max_element.text.replace(',', ''))
max_value

for i in range(1, max_value-(max_value-11)):
    counter = counter + 1
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.ID, 'collection-results-table')))
    html = driver.page_source

    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')

    # Find the table element by ID
    table = soup.find('table', {'id': 'collection-results-table'})

    # Extract the table rows
    rows = table.find_all('tr')


    for row in rows[1:]:
        cells = row.find_all('td')
        data.append({
            'Name': cells[1].text,
            'Birth Year': cells[2].text.strip(),
            'Arrival Date': cells[3].text.strip(),
            'Departure Port': cells[4].text.strip(),
            'Ethnicity/Nationality': cells[5].text.strip(),
            'Ship Name': cells[6].text.strip()
        })

    df = pd.DataFrame(data)

    print(counter)

    next_button = driver.find_element(By.CSS_SELECTOR, 'a.pagingNext')
    next_button.click()
    
    # Add random wait time between 1-3 seconds
    time.sleep(random.randint(1, 3))
    if counter % 10 == 0:
        df.to_csv(f'csvs/data_{port_entry}_{year}_{counter}.csv', index=False)

df.to_csv(f'csvs/data_{port_entry}_{year}_final.csv', index=False)

/var/folders/53/zn46v9s96111cklvxsj1blcw0000gn/T/ipykernel_70648/3107946731.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="./chromedriver_mac64/chromedriver")


1
2
3
4
5
6
7
8
9
10


In [5]:
df

NameError: name 'df' is not defined

In [ ]:
df